In [1]:
from collections import namedtuple
import csv
from itertools import islice

In [2]:
f_names = 'cars.csv','personal_info.csv'

In [3]:
def get_dialect(f_name):
    with open(f_name) as f:
        return csv.Sniffer().sniff(f.read(1000))
    
class FileParser:
    def __init__(self,f_name):
        self.f_name = f_name
        
    def __enter__(self):
        self._f = open(self.f_name,'r')
        self._reader = csv.reader(self._f,get_dialect(self.f_name))
        headers = map(lambda s :s.lower() ,next(self._reader))
        self._nt = namedtuple('Data',headers)
        return self   # it is a iterator
    
    
    def __exit__(self,exc_type,exc_value,exc_tb):
        self._f.close()
        return False
        
    def __iter__(self):
        return self
    
    
    def __next__(self):
        if self._f.closed:
            raise StopIteration
        else:
            return self._nt(*next(self._reader))  #next(self._reader) is a list, and self._nt doestnt expect a list , so unpack it with *

In [4]:
def parsed_data_iter(data_iter,nt):
    for row in data_iter:
        yield nt(*row)
        

In [5]:
from contextlib import contextmanager

In [6]:
@contextmanager
def parsed_data(f_name):
    f = open(f_name,'r')
    try:
        reader = csv.reader(f,get_dialect(f_name))
        headers = map(lambda s : s.lower(),next(reader))
        nt = namedtuple('Data',headers)
        yield parsed_data_iter(reader,nt)
    finally:
        f.close()
        

In [10]:
with parsed_data('personal_info.csv')as data:
    for row in islice(data,5):
        print(row)

Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
Data(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
Data(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')


In [23]:
@contextmanager
def parsed_data(f_name):
        
    f = open(f_name,'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(1000))
        f.seek(0)
        reader = csv.reader(f,dialect())
        headers = map(lambda s : s.lower(),next(reader))
        nt = namedtuple('Data',headers)
        yield (nt(*row) for row in reader)
    finally:
        f.close()
        

In [24]:
with parsed_data('personal_info.csv')as data:
    for row in islice(data,5):
        print(row)

Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
Data(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
Data(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')


In [25]:
with parsed_data('cars.csv')as data:
    for row in islice(data,5):
        print(row)

Data(car='Chevrolet Chevelle Malibu', mpg='18.0', cylinders='8', displacement='307.0', horsepower='130.0', weight='3504.', acceleration='12.0', model='70', origin='US')
Data(car='Buick Skylark 320', mpg='15.0', cylinders='8', displacement='350.0', horsepower='165.0', weight='3693.', acceleration='11.5', model='70', origin='US')
Data(car='Plymouth Satellite', mpg='18.0', cylinders='8', displacement='318.0', horsepower='150.0', weight='3436.', acceleration='11.0', model='70', origin='US')
Data(car='AMC Rebel SST', mpg='16.0', cylinders='8', displacement='304.0', horsepower='150.0', weight='3433.', acceleration='12.0', model='70', origin='US')
Data(car='Ford Torino', mpg='17.0', cylinders='8', displacement='302.0', horsepower='140.0', weight='3449.', acceleration='10.5', model='70', origin='US')
